# Определение наиболее выгодного региона нефтедобычи

В данной исследовательской работы мы исследуем несколько регионов и определим целесообразность разработки в них месторождений нефти.

**Цели исследования:**
1. На основании имеющихся данных построить предсказательную модель, которая отразит текущие запасы нефти в регионе.
2. Рассчитать минимальный средний запас нефти в регионах для их безубыточной разработки.
3. Рассчитать прибыль и риски по каждому региону на основании предсказанных моделью данных.

**Ход исследования:**
1. Загрузим и подготовим данные.
2. Обучим предсказательную модель на подготовленных данных.
3. Произведем подготовку к расчету прибыли и рисков.
4. Рассчитаем среднюю прибыль, 90% доверительный интервал прибыли и риски по каждому из регионов.

## Обзор данных

Испортируем необходимые для исследования библиотеки и загрузим имеющиеся данные по кажому региону.

In [1]:
# Импорт стандартных библиотек
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
try:
    geo_df_0 = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\'
                           'Data Science\\Обучение в Яндекс\\'
                           '9. Машинное обучение в бизнесе\\Проект\\geo_data_0.csv')
    geo_df_1 = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\'
                           'Data Science\\Обучение в Яндекс\\'
                           '9. Машинное обучение в бизнесе\\Проект\\geo_data_1.csv')
    geo_df_2 = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\'
                           'Data Science\\Обучение в Яндекс\\'
                           '9. Машинное обучение в бизнесе\\Проект\\geo_data_2.csv')
except:
    geo_df_0 = read_csv('/datasets/geo_data_0.csv')
    geo_df_1 = read_csv('/datasets/geo_data_1.csv')
    geo_df_2 = read_csv('/datasets/geo_data_2.csv')

In [3]:
geo_df_0.info()
geo_df_0.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo_df_1.info()
geo_df_1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
geo_df_2.info()
geo_df_2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Вывод

Из загруженных файлов видно, что у нас есть 100 тыс. различных данных геологоразведки по 3-м регионам. Данные состоят из признаков, характеризующих отдельно взятые скважины в уже исследованных регионах, и ключевых показателей объема запаса нефти в каждой из них.

## Предобработка данных

Предобработаем данные.
В данных отсутствуют пропуски, и значения признаков каждой из скважин сбалансированы, поэтому заменя пропусков и сэмплинг из предобработки исключим.

Проверим данные на наличие полных дубликатов.

In [6]:
display(geo_df_0.duplicated().sum(), geo_df_1.duplicated().sum(), geo_df_2.duplicated().sum())

0

0

0

Полные дубликаты в данных отсутствуют.

Проверим данные на наличие дубликов по id скважин.

In [7]:
display(geo_df_0.duplicated(subset=['id']).sum(),
        geo_df_1.duplicated(subset=['id']).sum(),
        geo_df_2.duplicated(subset=['id']).sum())

10

4

4

В данных есть дубликаты скважин.
Однако:
 * их очень маленькое количество;
 * все остальные признаки скважин отличаются.
В связи с изложенным мы оставим дубликаты скважин без изменений.

### Вывод

Данные достаточно чистые, их предобработка не требуется. Приступим к обучению моделей.

## Обучение и проверка модели

Разобъем данные на предсказательные признаки и ключевой признак по каждому региону.

In [8]:
features_0 = geo_df_0.drop(['id', 'product'], axis=1)
target_0 = geo_df_0['product']

features_1 = geo_df_1.drop(['id', 'product'], axis=1)
target_1 = geo_df_1['product']

features_2 = geo_df_2.drop(['id', 'product'], axis=1)
target_2 = geo_df_2['product']

display(features_0.shape, target_0.shape,
        features_1.shape, target_1.shape,
        features_2.shape, target_2.shape)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

Разобъем данные на обучающую и валидационную выборки по каждому региону.

In [9]:
state = np.random.RandomState(12345)

# Разбивка данных на обучающую и валидационную выборки
(features_train_0, features_valid_0,
 target_train_0, target_valid_0) = train_test_split(features_0, target_0,
                                                    test_size=.25, random_state=state)
(features_train_1, features_valid_1,
 target_train_1, target_valid_1) = train_test_split(features_1, target_1,
                                                    test_size=.25, random_state=state)
(features_train_2, features_valid_2,
 target_train_2, target_valid_2) = train_test_split(features_2, target_2,
                                                    test_size=.25, random_state=state)

display(features_train_0.shape, target_train_0.shape,
        features_valid_0.shape, target_valid_0.shape)
display(features_train_1.shape, target_train_1.shape,
        features_valid_1.shape, target_valid_1.shape)
display(features_train_2.shape, target_train_2.shape,
        features_valid_2.shape, target_valid_2.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

(75000, 3)

...

(25000,)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

Для обучения выбрана модель логистической регрессии, как самая предсказуемая в данном случае.

Обучим модели на данных каждого региона.

In [10]:
# Модель для Региона № 1
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

result_0 = mean_squared_error(target_valid_0, predictions_valid_0) ** .5
# Расчет среднего запаса сырья по региону
average_stock_materials_0 = predictions_valid_0.mean()

display(f'RMSE: {result_0:.2f}')
display(f'Средний запас сырья по региону: {average_stock_materials_0:.2f}')

'RMSE: 37.58'

'Средний запас сырья по региону: 92.59'

In [11]:
# Модель для Региона № 2
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

result_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** .5
# Расчет среднего запаса сырья по региону
average_stock_materials_1 = predictions_valid_1.mean()

display(f'RMSE: {result_1:.2f}')
display(f'Средний запас сырья по региону: {average_stock_materials_1:.2f}')

'RMSE: 0.89'

'Средний запас сырья по региону: 68.77'

In [12]:
# Модель для Региона № 1
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

result_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** .5
# Расчет среднего запаса сырья по региону
average_stock_materials_2 = predictions_valid_2.mean()

display(f'RMSE: {result_2:.2f}')
display(f'Средний запас сырья по региону: {average_stock_materials_2:.2f}')

'RMSE: 39.96'

'Средний запас сырья по региону: 95.09'

### Вывод

Модели обучены.

Наибольший запас сырья показывает регион № 3 с показателем 94.97 тыс. тонн. нефти. Вторым по показателям запаса нефти идет регион № 1.

В регионе № 2 самые маленькие запасы, однако корень из среднеквадратичной ошибки по предсказаниям в данном регионе стремится к нулю, что является неплохим показателем качества предсказаний второй модели.

## Подготовка к расчёту прибыли

Запишем ключевые значения для расчета прибыли и рисков в константы для удобства.

In [13]:
RESEARCH_POINTS = 500  # Количество скважин для анализа, шт.
BEST_POINTS = 200  # Количество выбираемых скважин, шт.
INVESTMENT = 10_000_000_000  # Инвестиции на разработку региона, млрд. руб.
BARREL_PRICE = 450 * 1000  # Доход с каждой единицы продукта, руб.
PROBABILITY_OF_LOSS = 2.5  # Вероятность убытка, %

Рассчитаем минимальный средний объем сырья для безубыточной разработки регионов.

In [14]:
# Расчет общегоо объема сырья для безубыточной разработки
min_total_materials_by_region = INVESTMENT / (BARREL_PRICE)
# Расчет среднего минимального объема сырья для безубыточной разработки
required_material_stocks = INVESTMENT / (BARREL_PRICE) / BEST_POINTS

display(f'Общий объем сырья выбранных скважин для их безубыточной разработки:',
        f'{min_total_materials_by_region:.2f} тыс. тонн баррелей')
display('Средний объем сырья в каждой скважине для их безубыточной разработки:',
        f'{required_material_stocks:.2f} тыс. тонн баррелей')

'Общий объем сырья выбранных скважин для их безубыточной разработки:'

'22222.22 тыс. тонн баррелей'

'Средний объем сырья в каждой скважине для их безубыточной разработки:'

'111.11 тыс. тонн баррелей'

### Вывод

Из расчетов видно, что общий объем сырья выбранных скважин для их безубыточной разработки должен составлять не менее 22 222.22 тыс. тонн баррелей, а средний объем сырья в каждой скважине для их безубыточной разработки должен составлять не менее 111.11 тыс. тонн баррелей.

## Расчёт прибыли и рисков

Произведем расчет прибыли и рисков.

Для начала переведем предсказанные значения к формату Series, а индексы ключевых признаков сбросим для избежания ошибок в коде.

In [15]:
# Преобразование numpy.ndarray в pd.Series
predictions_valid_0 = pd.Series(predictions_valid_0)
predictions_valid_1 = pd.Series(predictions_valid_1)
predictions_valid_2 = pd.Series(predictions_valid_2)

# Дроп индексов ключевых показателей валидационной выборки (для ичключения KeyError)
target_valid_0 = target_valid_0.reset_index(drop=True)
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)

Напишем функции для расчета прибыли и рисков.

In [16]:
# Функция расчета прибыли
def revenue(predict_subsample, target_subsample, count=BEST_POINTS):
    predict_sorted = predict_subsample.sort_values(ascending=False)
    selected = target_subsample[predict_sorted.index][:count]
    value = (selected.sum() * BARREL_PRICE - INVESTMENT) / 1000000000
    return value


# Boostrap функция, расчет ключевых показателей
def bootstrap(predictions_valid, target):
    values = []
    for i in range(1000):
        target_subsample = target.sample(
            n=RESEARCH_POINTS, replace=True, random_state=state)
        predict_subsample = predictions_valid[target_subsample.index]
        # Передача значений ф-ции revenue
        values.append(revenue(predict_subsample, target_subsample))
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    values_mean = values.mean()
    values_loss = [num for num in values if num < 0]  # Список убытков
    # Доля убытков в общем объеме значений прибыли
    risk = len(values_loss) / len(values)
    return lower, upper, values_mean, risk


calculation_0 = bootstrap(predictions_valid_0, target_valid_0)
calculation_1 = bootstrap(predictions_valid_1, target_valid_1)
calculation_2 = bootstrap(predictions_valid_2, target_valid_2)

calculation_list = [calculation_0, calculation_1, calculation_2]
for i in range(len(calculation_list)):
    display(f'Регион {i}:')
    display(f'\t Средняя прибыль: {calculation_list[i][2]:.2f} млрд. руб.')
    display(f'\t Доверительный интервал: {calculation_list[i][0]:.2f}',
            f'- {calculation_list[i][1]:.2f} млрд. руб.')
    display(f'\t Риск убытков: {calculation_list[i][3]:.2%} \n')

'Регион 0:'

'\t Средняя прибыль: 0.42 млрд. руб.'

'\t Доверительный интервал: -0.08'

'- 0.96 млрд. руб.'

'\t Риск убытков: 4.80% \n'

...

'Регион 2:'

'\t Средняя прибыль: 0.38 млрд. руб.'

'\t Доверительный интервал: -0.14'

'- 0.89 млрд. руб.'

'\t Риск убытков: 7.40% \n'

### Вывод

Из рассчитанной прибыли и рисков можно сделать следующие выводы:
1. Регион № 1:
   
   Средняя прибыль по региону составляет 420 млн. руб., диапазон вириации прибыли составляет от убытка в 80 млн. руб. до прибыли в 960 млн. руб. Риск получить убыток составляет 4.80%.
2. Регион № 2:
   
   Средняя прибыль по региону составляет 510 млн. руб., диапазон вириации прибыли составляет от прибыли в 110 млн. руб. до прибыли в 930 млн. руб. Риск получить убыток составляет 0.60%.
3. Регион № 3:
   
   Средняя прибыль по региону составляет 380 млн. руб., диапазон вириации прибыли составляет от убытка в 140 млн. руб. до прибыли в 890 млн. руб. Риск получить убыток составляет 7.40%.

## Общий вывод

Нами прововеден анализ нескольких нефтесодержащих регионов и определена целесообразность разработки нефтяных месторождений в каждом из них.

По итогам анализа можно сделать вывод, что наиболее прибыльным регионом является регион под № 2 со средней прибылью по региону 510 млн. руб. и с безубыточным доверительным интервалом от 110 до 930 млн. руб. Риск получить убыток в данном регионе составляет всего 0.60%, что значительно ниже инвестиционной оценки риска в 2.5%. Иные регионы имеют высокий риск получения убытков.

Исходя из анализа для разработки нефтяных месторождений целесообразно выбрать регион под № 2.